In [26]:
import firebase_admin
from firebase_admin import credentials, db
from unidecode import unidecode
import ipywidgets as widgets
from IPython.display import display, clear_output

# Check if Firebase app is already initialized
if not firebase_admin._apps:
    # Initialize the Firebase app with the certificate and the URL of the Realtime Database
    cred = credentials.Certificate("credentials.json")
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://inf857---tp1---exo1---imc-default-rtdb.firebaseio.com/'
    })

# Create output widget
output = widgets.Output()

# Function to write data in firebase
def write_data(name, weight, height):
    with output:
        clear_output()
        # Validation checks
        if len(name) < 1:
            print("Error: Name must have at least one character.")
            return
        try:
            weight = float(weight)
            height = float(height)
        except ValueError:
            print("Error: Weight and height must be numeric values.")
            return
        ref = db.reference('users')
        imc = round(weight / (height / 100) ** 2, 2) # Calculate IMC
        # write data
        ref.push({
            'name': unidecode(name).lower(), # standardize string
            'weight': weight,
            'height': height,
            'imc': imc
        })
        print(f"Data for {name} added successfully! IMC: {imc}")

# Function to read data
def read_data(search_name):
    with output:
        clear_output()
        search_name = unidecode(search_name).lower() # standardize string
        ref = db.reference('users')
        data = ref.get() # get data from child 'users' in firebase

        # Search in the data
        find = False
        if data:
            for _, value in data.items():
                if value['name'] == search_name:
                    # We may find more than one match, in this case we print all of them
                    print(f"{value['name']}: Weight = {value['weight']} kg, Height = {value['height']} cm, IMC = {value['imc']}")
                    find = True
        if not find:
            print("No data found for this name.")

# Function to handle add data button click
def on_add_data_clicked(b):
    name = name_input.value
    weight = weight_input.value
    height = height_input.value
    write_data(name, weight, height)

# Function to handle read data button click
def on_read_data_clicked(b):
    search_name = search_input.value
    read_data(search_name)

# Create input widgets
name_input = widgets.Text(placeholder='Your name')
weight_input = widgets.Text(placeholder='Your weight (kg)')
height_input = widgets.Text(placeholder='Your height (cm)')
search_input = widgets.Text(placeholder='Name to search')

# Create buttons
add_data_button = widgets.Button(description="Add Data")
read_data_button = widgets.Button(description="Read Data")

# Set up button click events
add_data_button.on_click(on_add_data_clicked)
read_data_button.on_click(on_read_data_clicked)

# Display widgets
display(name_input, weight_input, height_input, add_data_button)
display(search_input, read_data_button)
display(output)


Text(value='', placeholder='Your name')

Text(value='', placeholder='Your weight (kg)')

Text(value='', placeholder='Your height (cm)')

Button(description='Add Data', style=ButtonStyle())

Text(value='', placeholder='Name to search')

Button(description='Read Data', style=ButtonStyle())

Output()